In [1]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/My Drive/5001proj")

Mounted at /content/gdrive


In [ ]:
!pip install ultralytics

In [2]:
import json
import argparse
from PIL import Image


In [ ]:
from ultralytics import YOLO

In [3]:
json_path ='./data/train_coco.json'
save_path ='./data/labels/train_aug'

In [4]:
def convert(size, bbox):
    x = bbox[0] + bbox[2] / 2.0
    y = bbox[1] + bbox[3] / 2.0
    width = bbox[2]
    height = bbox[3]
    return (x, y, width, height)

In [5]:
#COCO转yolo
data = json.load(open(json_path, 'r'))

id_map = {}
with open(os.path.join(save_path, 'classes.txt'), 'w') as f:
    for i, category in enumerate(data['categories']):
        f.write(f"{category['name']}\n")
        id_map[category['id']] = i



In [7]:
path_record = open(os.path.join(save_path, 'train.txt'), 'w')
for img in data['images']:
    head, tail = os.path.splitext(img["file_name"])
    ana_txt_name = head + ".txt"
    f_txt = open(os.path.join(save_path, ana_txt_name), 'w')
    for annotation in data['annotations']:
        if annotation['image_id'] == img["id"]:
            bbox = convert((img["width"], img["height"]), annotation["bbox"])
            f_txt.write("%s %s %s %s %s"%(id_map[annotation["category_id"]],bbox[0],bbox[1],bbox[2],bbox[3]))
            counter=0
            for i in range(len(annotation["keypoints"])):
                if annotation["keypoints"][i] == 2 or annotation["keypoints"][i] == 1 or annotation["keypoints"][i] == 0:
                    f_txt.write("%s"%format(annotation["keypoints"][i],'3f'))
                    counter=0
                else:
                    if counter==0:
                        f_txt.write("%s"%(annotation["keypoints"][i]/img["width"]))
                    else:
                        f_txt.write("%s"%(annotation["keypoints"][i]/img["height"]))
                    counter+=1
    f_txt.write("\n")
    f_txt.close()

    path_record.write('./data/images_original/%s.jpg\n' % (head))
path_record.close()

In [ ]:
#移动图片

import shutil
goal_path="./data/images/train_aug/"

f = open("./data/label/train.txt","r")
lines = f.readlines()


In [ ]:
len(lines)

168

In [ ]:
for line in lines:
  #print(line[23:-1])
  shutil.move(line[:-1],goal_path+line[23:-1])

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
from ultralytics import YOLO

In [ ]:
model = YOLO('yolov8n-pose.pt')

results = model.train(data='/content/gdrive/My Drive/5001proj/config.yaml', epochs=100, imgsz=640)

In [ ]:

model = YOLO('yolov8n-pose.pt')  # load a pretrained model (recommended for training)

results = model.train(data='/content/gdrive/My Drive/5001proj/config.yaml', epochs=100, imgsz=640, hsv_h=0,hsv_s=0,hsv_v=0,degrees=45,translate=0,scale=0,shear=30,fliplr=0,mosaic=1)

# results = model.train(data='/content/gdrive/My Drive/5001proj/config.yaml', epochs=100, imgsz=640, degrees=45,translate=0,scale=0,shear=30,fliplr=0,mosaic=1,mixup=0.1,perspective=0.0001)


Ultralytics YOLOv8.0.219 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
engine/trainer: task=pose, mode=train, model=yolov8n-pose.pt, data=/content/gdrive/My Drive/5001proj/config.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train11, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes

train: Scanning /content/gdrive/My Drive/5001proj/data/labels/train.cache... 168 images, 0 backgrounds, 0 corrupt: 100%|██████████| 168/168 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/gdrive/My Drive/5001proj/data/labels/val.cache... 42 images, 0 backgrounds, 0 corrupt: 100%|██████████| 42/42 [00:00<?, ?it/s]


Plotting labels to runs/pose/train11/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 63 weight(decay=0.0), 73 weight(decay=0.0005), 72 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/pose/train11
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


...

    100/100      2.45G     0.8238     0.8864    0.09238     0.3891      1.411          8        640: 100%|██████████| 11/11 [00:23<00:00,  2.15s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.37it/s]

                   all         42         42      0.999          1      0.995      0.638      0.999          1      0.995      0.705



100 epochs completed in 0.840 hours.
Optimizer stripped from runs/pose/train11/weights/last.pt, 7.0MB
Optimizer stripped from runs/pose/train11/weights/best.pt, 7.0MB

Validating runs/pose/train11/weights/best.pt...
Ultralytics YOLOv8.0.219 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
YOLOv8n-pose summary (fused): 187 layers, 3379496 parameters, 0 gradients, 9.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:08<00:00,  4.21s/it]


                   all         42         42      0.999          1      0.995      0.638      0.999          1      0.995      0.705
Speed: 0.2ms preprocess, 2.6ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/pose/train11


In [ ]:

model = YOLO('/content/gdrive/My Drive/5001proj/best.pt')

results = model('/content/gdrive/My Drive/5001proj/1.jpg')

# Show the results
for r in results:
    im_array = r.plot(kpt_radius=20)
    im = Image.fromarray(im_array[..., ::-1])
    im.show()
    im.save('result1.jpg')

In [ ]:
!yolo pose predict model="/content/gdrive/My Drive/5001proj/best.pt" source='/content/gdrive/My Drive/5001proj/v.mp4' show=True save=True